In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
print(__doc__)
import seaborn as sns
import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import cPickle as pkl
import statsmodels.api as sm
from signale.tools import locate
import signale
from os import path

/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Automatically created module for IPython interactive environment


In [4]:
sns.set_style('white')
sns.set_context("poster",font_scale=1.7)
leftRight = sns.color_palette(['#f9ba32','#426e86'])
dgc = sns.color_palette(['#0f1f38','#c5001A','#1b45ba'])
sns.palplot(dgc)
#lrC = sns.xkcd_palette(leftRight)

In [5]:
sigSigPths = []
for item in locate('awakeReplays-*RO.db','/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/'):
    dbPth = path.join(item[0],item[1])
    awro = pd.read_pickle(dbPth)
    sigRO = awro[(awro.Lpv<0.05)|(awro.Rpv<0.05)]
    sigSig = sigRO[sigRO.tSub>10]
    if sigSig.shape[0] and dbPth.find('Rat616')<0:
        sigSigPths.append(path.join(item[0],item[1]))
    

In [7]:
for dbName in sigSigPths[:]:
    group = dbName.split('/')[-4]
    dday  = dbName.split('/')[-1].split('-')[-2]
    anID  = dbName.split('/')[-1].split('-')[-3]
    RtName = [path.join(ijk[0],ijk[1])for ijk in locate('Rt*.nparr','/home/chenani/ownCloud/') if (path.join(ijk[0],ijk[1]).find('Rat'+anID)>0 and path.join(ijk[0],ijk[1]).find(dday)>0)][0]
    LtName = [path.join(ijk[0],ijk[1])for ijk in locate('Lt*.nparr','/home/chenani/ownCloud/') if (path.join(ijk[0],ijk[1]).find('Rat'+anID)>0 and path.join(ijk[0],ijk[1]).find(dday)>0)][0]
    spkName = [path.join(item[0],item[1])for item in locate('placeCells.spk','/home/chenani/dataWork/ali/MECLesion_SleepData/') 
               if ((path.join(item[0],item[1]).find((dday[0]+'ay'+dday[-1]))>0) and (path.join(item[0],item[1]).find(anID)>0))][0]
    trjName = [path.join(item[0],item[1])for item in locate('linearTrack_Trajectory.traj','/home/chenani/dataWork/ali/MECLesion_SleepData/') 
               if ((path.join(item[0],item[1]).find((dday[0]+'ay'+dday[-1]))>0) and (path.join(item[0],item[1]).find(anID)>0))][0]
    ########
    awro = pkl.load(open(dbName))
    Rtemp = pkl.load(open(RtName))
    Ltemp = pkl.load(open(LtName))
    spk = pkl.load(open(spkName))
    trj = pkl.load(open(trjName))
    sigRO = awro[(awro.Lpv<0.05)|(awro.Rpv<0.05)]
    #sigSig = awro[(awro.Lcorr**2<0.05)&(awro.Rcorr**2<0.05)]
    df = sigRO[sigRO.tSub>14]
    #df = sigSig[sigSig.tSub>14]
    k = 0
    for idx,row in df.iterrows():
        f,(ax,bx) = pl.subplots(2,1,figsize = [4,12],sharex=True)
        zug = spk.time_slice(row.ti-1e0,row.tf+1e0)
        zug.rasterPlot(temp=Ltemp,lw=4,clr=leftRight[0],meanClr='k',fig=f,ax=ax,activeonly=True,yticks=False)# dgc[1]
        zug.rasterPlot(temp=Rtemp,lw=4,clr=leftRight[1],meanClr='k' ,fig=f,ax=bx,activeonly=True,yticks=False)#'#ff0038'
        bx.set_xticks([row.ti,row.tf])
        bx.set_xticklabels([0,int(row.tf-row.ti)])
        bx.set_xlabel('time (ms)')
        bx.set_xlim(zug.t_start,zug.t_stop)
        axYmax = ax.get_ylim()
        bxYmax = bx.get_ylim()
        ax.set_ylim(axYmax[0],axYmax[1]+1)
        bx.set_ylim(bxYmax[0],bxYmax[1]+1)
        ax.text(round(zug.t_stop-3*(zug.t_stop-zug.t_start)/10),int(axYmax[1]+0.5),'r = '+str(round(row.Lcorr,2)),fontsize=22,ha='center', va='center')#, transform=ax.transAxes)
        bx.text(round(zug.t_stop-3*(zug.t_stop-zug.t_start)/10),int(bxYmax[1]+0.5),'r = '+str(round(row.Rcorr,2)),fontsize=22,ha='center', va='center')#, transform=bx.transAxes)
        f.tight_layout()
        f.savefig('/home/chenani/ownCloud/Plots/Lesion-Project/Rank-Order/replay-rasters/'+anID+'-'+dday+'-'+group+'-awake999-'+str(idx)+'.eps')
        k+=1
    
    
#axarr[1].plot(range(10),'r--')

In [7]:
#df = sigRO[sigRO.tSub>4]
df = sigSig
k = 0
for idx,row in df.iterrows():
    f,(ax,bx) = pl.subplots(2,1,figsize = [4,12],sharex=True)
    zug = spk.time_slice(row.ti-1e0,row.tf+1e0)
    zug.rasterPlot(temp=Ltemp,lw=4,clr=leftRight[0],fig=f,ax=ax,activeonly=True,yticks=True)
    zug.rasterPlot(temp=Rtemp,lw=4,clr=leftRight[1],fig=f,ax=bx,activeonly=True,yticks=True)
    bx.set_xticks([row.ti,row.tf])
    bx.set_xticklabels([0,int(row.tf-row.ti)])
    bx.set_xlabel('time (ms)')
    f.tight_layout()
    #f.savefig()
    k+=1
    
    
#axarr[1].plot(range(10),'r--')

In [8]:
df

,ti,tf,tp,tSub,lSub,rSub,Lcorr,Rcorr,Lpv,Rpv,animal_ID


In [147]:
[path.join(item[0],item[1])for item in locate('placeCells.spk','/home/chenani/dataWork/ali/MECLesion_SleepData/') 
 if ((path.join(item[0],item[1]).find((dday[0]+'ay'+dday[-1]))>0) and (path.join(item[0],item[1]).find(anID)>0))]

['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3906/LinearTrack_06-02-16_Day3/parrentData/placeCells.spk']

In [142]:
(dday[0]+'ay'+dday[-1])

'Day3'